In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

print("init model done")

init model done


In [25]:
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("set vars and device done")

set vars and device done


In [26]:
transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
  datasets.MNIST('../data', train=True, download=True, 
                 transform=transform), 
    batch_size = batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
                 transform=transform), 
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [27]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay = 0.01 )
#L2 Regularization

In [28]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:
for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model.pt')

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290312
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.970073
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.409932
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.489804
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.253488

Test set: Average loss: 0.3911, Accuracy: 8779/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.407881
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.258478
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.370356
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.241571
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.609459

Test set: Average loss: 0.6661, Accuracy: 7924/10000 (79%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.949840
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.218495
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.469401
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.191604
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.357672

Test set: Average loss: 0.3096, Accuracy: 9080/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.236626
Train Epoch: 4 [12800/60000 (21%)]	Lo